------

 <center><h1>🐦 Extraer Tuits 🐦 </center> 

    
<br><b>Creador:</b> Norma Yuliana Cala Martínez
<br><b>Email: </b> normisss11@gmail.com
<br><b>Fecha creacion:</b> Noviembre 12, 2020

### Punto 1: Extraer Tuits
Extraer mínimo 100 tuits de Twitter usando la API


# Autenticarse con Twitter

In [34]:
#!pip install tweepy
import pandas as pd 
import tweepy

In [35]:
# Leer keys
keys = pd.read_csv("Twitter_Keys_Text.csv", header=None)
keys = dict(zip(keys[0],keys[1]))

In [36]:
# Autorizar a través API
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
api.update_status('La mejor clase es NLP.')

## Datos de la Red social de Twitter. 

In [39]:
# Tema seleccionado son las eliminatorias al mundial de futbol.  ⚽
tweets = tweepy.Cursor(api.search,
                       q=['eliminatorias -RT'],
                       lang='es',
                       geocode=f'9.304744,-75.397778,45mi',
                       tweet_mode='extended',
                       include_rts=False).items(200)

In [40]:
t = []

for tweet in tweets:
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append("colombia")
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/status/{tweet.id}")
    
    t.append(temp)

In [41]:
t

[['¡ATENCIÓN! #DEPORTES \nEliminatorias Sudamericanas: la Selección de Chile venció 2-0 a Perú en la fecha 3\nConozcas más detalles aquí en #ChicaNoticias \nhttps://t.co/bYsjJaBwIQ',
  datetime.datetime(2020, 11, 14, 1, 58, 28),
  'colombia',
  'Córdoba, Colombia',
  1327430669589471233,
  'https://twitter.com/i/web/status/1327430669589471233'],
 ['Observando a las otras selecciones en las eliminatorias, juegan con una intensidad con mucha presión a los rivales, algo que #Colombia carece; Van a estar bien complicado para los nuestros. Hoy jugaron sin alma, una falta de respeto al país futbolero. https://t.co/8EdtfG923D',
  datetime.datetime(2020, 11, 14, 0, 53, 39),
  'colombia',
  'Montería, Colombia',
  1327414355143626752,
  'https://twitter.com/i/web/status/1327414355143626752'],
 ['Primer partido que me veo de Colombia en las eliminatorias y pierde 😒🙄',
  datetime.datetime(2020, 11, 14, 0, 2, 18),
  'colombia',
  'Sahagún, Colombia',
  1327401433185083398,
  'https://twitter.com/i

In [42]:
data = pd.DataFrame(t)
data.columns = ['tweet', 'date', 'city', 'user_location', 'id', 'link']
data.head()

,tweet,date,city,user_location,id,link
0,¡ATENCIÓN! #DEPORTES \nEliminatorias Sudameric...,2020-11-14 01:58:28,colombia,"Córdoba, Colombia",1327430669589471233,https://twitter.com/i/web/status/1327430669589...
1,Observando a las otras selecciones en las elim...,2020-11-14 00:53:39,colombia,"Montería, Colombia",1327414355143626752,https://twitter.com/i/web/status/1327414355143...
2,Primer partido que me veo de Colombia en las e...,2020-11-14 00:02:18,colombia,"Sahagún, Colombia",1327401433185083398,https://twitter.com/i/web/status/1327401433185...
3,#COLURU Eliminatorias Sudamericanas: Colombia ...,2020-11-13 23:52:46,colombia,lorica córdoba colombia,1327399035356008448,https://twitter.com/i/web/status/1327399035356...
4,!MUCHA ATENCIÓN! #NACIONAL #DEPORTES\n¡Nos com...,2020-11-13 23:27:26,colombia,"Córdoba, Colombia",1327392657728004096,https://twitter.com/i/web/status/1327392657728...


In [43]:
# Exportar datos.
data.to_csv("datos13112020.csv", index=False)

##  Pre-procesamiento para Twitter 
⚽

In [10]:
data = data[['tweet', 'date', 'city']]
data.head()

,tweet,date,city
0,¡ATENCIÓN! #DEPORTES \nEliminatorias Sudameric...,2020-11-14 01:58:28,colombia
1,Observando a las otras selecciones en las elim...,2020-11-14 00:53:39,colombia
2,Primer partido que me veo de Colombia en las e...,2020-11-14 00:02:18,colombia
3,#COLURU Eliminatorias Sudamericanas: Colombia ...,2020-11-13 23:52:46,colombia
4,!MUCHA ATENCIÓN! #NACIONAL #DEPORTES\n¡Nos com...,2020-11-13 23:27:26,colombia


### Quitar links, menciones, hashtags, Quitar emojis

In [11]:
import re

pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweet']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
             
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag,"")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention,"")
        
    return text, links, hashtags, mentions

In [12]:
data[['clean', 'links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')
data.head()

,tweet,date,city,clean,links,hashtags,mentions
0,¡ATENCIÓN! #DEPORTES \nEliminatorias Sudameric...,2020-11-14 01:58:28,colombia,¡ATENCIÓN! \nEliminatorias Sudamericanas: la ...,[https://t.co/bYsjJaBwIQ],"[#DEPORTES, #ChicaNoticias]",[]
1,Observando a las otras selecciones en las elim...,2020-11-14 00:53:39,colombia,Observando a las otras selecciones en las elim...,[https://t.co/8EdtfG923D],[#Colombia],[]
2,Primer partido que me veo de Colombia en las e...,2020-11-14 00:02:18,colombia,Primer partido que me veo de Colombia en las e...,[],[],[]
3,#COLURU Eliminatorias Sudamericanas: Colombia ...,2020-11-13 23:52:46,colombia,Eliminatorias Sudamericanas: Colombia cae en ...,[https://t.co/775sgBhOyM],[#COLURU],[]
4,!MUCHA ATENCIÓN! #NACIONAL #DEPORTES\n¡Nos com...,2020-11-13 23:27:26,colombia,!MUCHA ATENCIÓN! \n¡Nos complicamos! Así qued...,[https://t.co/HlwwPYI0Ie],"[#NACIONAL, #DEPORTES, #CHN]",[]


In [13]:
#!pip install emoji
import regex
import emoji

In [14]:
def get_emojis(text):
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return emoji_list
data['emojis'] = data['clean'].apply(lambda text: get_emojis(text))

In [15]:
data.head()

,tweet,date,city,clean,links,hashtags,mentions,emojis
0,¡ATENCIÓN! #DEPORTES \nEliminatorias Sudameric...,2020-11-14 01:58:28,colombia,¡ATENCIÓN! \nEliminatorias Sudamericanas: la ...,[https://t.co/bYsjJaBwIQ],"[#DEPORTES, #ChicaNoticias]",[],[]
1,Observando a las otras selecciones en las elim...,2020-11-14 00:53:39,colombia,Observando a las otras selecciones en las elim...,[https://t.co/8EdtfG923D],[#Colombia],[],[]
2,Primer partido que me veo de Colombia en las e...,2020-11-14 00:02:18,colombia,Primer partido que me veo de Colombia en las e...,[],[],[],"[😒, 🙄]"
3,#COLURU Eliminatorias Sudamericanas: Colombia ...,2020-11-13 23:52:46,colombia,Eliminatorias Sudamericanas: Colombia cae en ...,[https://t.co/775sgBhOyM],[#COLURU],[],[]
4,!MUCHA ATENCIÓN! #NACIONAL #DEPORTES\n¡Nos com...,2020-11-13 23:27:26,colombia,!MUCHA ATENCIÓN! \n¡Nos complicamos! Así qued...,[https://t.co/HlwwPYI0Ie],"[#NACIONAL, #DEPORTES, #CHN]",[],[]


### Punto 2: Hacer una visualización sobre esos tuits


In [16]:
#!pip install plotly

## Series de tiempo

In [17]:
data['fecha'] = data.date.dt.date
temp = pd.DataFrame(data.fecha.value_counts()).reset_index()
temp.columns = ['fecha', 'cnt']
temp = temp.sort_values('fecha')
temp

,fecha,cnt
6,2020-11-05,1
5,2020-11-07,2
2,2020-11-08,5
3,2020-11-09,3
7,2020-11-10,1
1,2020-11-12,10
0,2020-11-13,17
4,2020-11-14,3


In [22]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)

In [29]:
trace = go.Scatter(x=temp.fecha.values,
                   y=temp.cnt.values,
                   text=[f"Fecha: {f}<br>Tuits: {c}" for f,c in zip(temp.fecha.values,temp.cnt.values)],
                   hoverinfo='text',
                   mode='lines+markers',
                   name='Días',
                   line={'color':'#1DA1F2'})

layout = go.Layout(title='Número de tuits por día')

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

Barras

In [30]:
from collections import Counter

temp_emojis = pd.DataFrame(list(zip(Counter(sum(data.emojis.values,[])).keys(),Counter(sum(data.emojis.values,[])).values())))
temp_emojis.columns = ['emoji', 'cnt']
temp_emojis.sort_values('cnt', ascending=False, inplace=True)
temp_emojis.head()

,emoji,cnt
13,🚨,4
2,🎥,2
5,🇨🇴,2
6,🤩,2
8,👕,2


In [31]:
trace = go.Bar(x=temp_emojis.emoji.values[:15],
               y=temp_emojis.cnt.values[:15])

layout = go.Layout(title='Emojis más usados')

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)